In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('diabetes.csv')
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [3]:
df.corr()['Outcome']

Pregnancies                 0.221898
Glucose                     0.466581
BloodPressure               0.065068
SkinThickness               0.074752
Insulin                     0.130548
BMI                         0.292695
DiabetesPedigreeFunction    0.173844
Age                         0.238356
Outcome                     1.000000
Name: Outcome, dtype: float64

In [4]:
X = df.iloc[:,:-1].values
y = df.iloc[:,-1].values

In [5]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [6]:
X = scaler.fit_transform(X)

In [7]:
X.shape

(768, 8)

In [8]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=1)

In [68]:
import tensorflow
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense,Dropout

In [10]:
model = Sequential()
model.add(Dense(32,activation='relu',input_dim=8))
model.add(Dense(1,activation='sigmoid'))

model.compile(optimizer='Adam',loss='binary_crossentropy',metrics=['accuracy'])

C:\Users\himan\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [11]:
model.fit(X_train,y_train,batch_size=32,epochs=100,validation_data=(X_test,y_test))

Epoch 1/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 4s 36ms/step - accuracy: 0.4855 - loss: 0.7274 - val_accuracy: 0.6818 - val_loss: 0.6393
Epoch 2/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.6351 - loss: 0.6658 - val_accuracy: 0.7273 - val_loss: 0.5926
Epoch 3/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.6880 - loss: 0.6025 - val_accuracy: 0.7532 - val_loss: 0.5612
Epoch 4/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7118 - loss: 0.5706 - val_accuracy: 0.7403 - val_loss: 0.5386
Epoch 5/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.6865 - loss: 0.5549 - val_accuracy: 0.7403 - val_loss: 0.5210
Epoch 6/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7054 - loss: 0.5455 - val_accuracy: 0.7532 - val_loss: 0.5093
Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7282 - loss: 0.5050 - val_accuracy: 0.7597 - val_loss: 0.4987
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7550 - loss: 0.4894 - val_accuracy: 0.7727 - 

# now use of keras tuner 

In [12]:
pip install -q -U keras-tuner

Note: you may need to restart the kernel to use updated packages.


In [13]:
import keras_tuner as kt

In [14]:
def model_builder(hp):
  model = keras.Sequential()
    
  model.add(Dense(32,activation='relu',input_dim = 8))
  model.add(Dense(1,activation='sigmoid'))

  optimizer = hp.Choice('optimizer',values = ['adam','sgd','rmsprop','adadelta'])
    
  model.compile(optimizer = optimizer,
                loss='binary_crossentropy',
                metrics=['accuracy'])

  return model

In [19]:
tuner = kt.Hyperband(model_builder,
                    objective='val_accuracy',
                    max_epochs=10)

In [22]:
tuner.search(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

Trial 4 Complete [00h 00m 04s]
val_accuracy: 0.7207792401313782

Best val_accuracy So Far: 0.7207792401313782
Total elapsed time: 00h 00m 23s


In [23]:
tuner.get_best_hyperparameters()[0].values

{'optimizer': 'rmsprop',
 'tuner/epochs': 2,
 'tuner/initial_epoch': 0,
 'tuner/bracket': 2,
 'tuner/round': 0}

In [26]:
model = tuner.get_best_models(num_models=1)[0]

C:\Users\himan\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\himan\anaconda3\Lib\site-packages\keras\src\saving\saving_lib.py:415: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 6 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [27]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 32)                  │             288 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │              33 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 321 (1.25 KB)

 Trainable params: 321 (1.25 KB)

 Non-trainable params: 0 (0.00 B)

In [28]:
model.fit(X_train,y_train,batch_size=32,epochs=100,initial_epoch=6,validation_data=(X_test,y_test))

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 33ms/step - accuracy: 0.7322 - loss: 0.6105 - val_accuracy: 0.7273 - val_loss: 0.5538
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7673 - loss: 0.5582 - val_accuracy: 0.7662 - val_loss: 0.5275
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7533 - loss: 0.5461 - val_accuracy: 0.7792 - val_loss: 0.5077
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7693 - loss: 0.5134 - val_accuracy: 0.7727 - val_loss: 0.4938
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7488 - loss: 0.5177 - val_accuracy: 0.7727 - val_loss: 0.4836
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7753 - loss: 0.4830 - val_accuracy: 0.7857 - val_loss: 0.4770
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7928 - loss: 0.4527 - val_accuracy: 0.7792 - val_loss: 0.4727
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7782 - loss: 0.4657 - val_accuracy: 0.79

In [31]:
def build_model(hp):
  model = Sequential()

  units = hp.Int('units',min_value = 8,max_value = 128,step = 8)

  model.add(Dense(units=units,activation='relu',input_dim = 8))
  model.add(Dense(1,activation='sigmoid'))

  model.compile(optimizer='rmsprop',loss= 'binary_crossentropy',metrics=['accuracy'])

  return model

In [35]:
tuner = kt.Hyperband(build_model,
                    objective='val_accuracy',
                    max_epochs=10,directory='mydir')

C:\Users\himan\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [36]:
tuner.search(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

Trial 21 Complete [00h 00m 05s]
val_accuracy: 0.6753246784210205

Best val_accuracy So Far: 0.8051947951316833
Total elapsed time: 00h 01m 34s


In [37]:
tuner.get_best_hyperparameters()[0].values

{'units': 80,
 'tuner/epochs': 4,
 'tuner/initial_epoch': 2,
 'tuner/bracket': 2,
 'tuner/round': 1,
 'tuner/trial_id': '0002'}

In [38]:
 model = tuner.get_best_models(num_models=1)[0]

In [39]:
model.fit(X_train,y_train,batch_size=32,epochs=100,initial_epoch=6,validation_data=(X_test,y_test))

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 3s 33ms/step - accuracy: 0.8007 - loss: 0.4701 - val_accuracy: 0.7857 - val_loss: 0.4872
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.7514 - loss: 0.4735 - val_accuracy: 0.7792 - val_loss: 0.4807
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7834 - loss: 0.4526 - val_accuracy: 0.7857 - val_loss: 0.4766
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7619 - loss: 0.4850 - val_accuracy: 0.7792 - val_loss: 0.4728
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7446 - loss: 0.4748 - val_accuracy: 0.7792 - val_loss: 0.4719
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7673 - loss: 0.4506 - val_accuracy: 0.7792 - val_loss: 0.4709
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7767 - loss: 0.4623 - val_accuracy: 0.7857 - val_loss: 0.4708
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7580 - loss: 0.4660 - val_accuracy: 0.7

In [43]:
def build_model(hp):
  model = Sequential()

  model.add(Dense(72,activation='relu',input_dim = 8))
      
  for i in range(hp.Int('num_layers',min_value = 1,max_value = 10)):
      model.add(Dense(72,activation='relu'))
      
  
  model.add(Dense(1,activation='sigmoid'))

  model.compile(optimizer='rmsprop',loss= 'binary_crossentropy',metrics=['accuracy'])

  return model

In [44]:
tuner = kt.Hyperband(build_model,
                    objective='val_accuracy',
                    max_epochs=3,directory='mydir',project_name='num_layers')

C:\Users\himan\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [45]:
tuner.search(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

Trial 6 Complete [00h 00m 07s]
val_accuracy: 0.7922077775001526

Best val_accuracy So Far: 0.8051947951316833
Total elapsed time: 00h 00m 37s


In [46]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 2,
 'tuner/epochs': 3,
 'tuner/initial_epoch': 1,
 'tuner/bracket': 1,
 'tuner/round': 1,
 'tuner/trial_id': '0002'}

In [48]:
model = tuner.get_best_models(num_models=1)[0]

In [49]:
model.fit(X_train,y_train,epochs=100,initial_epoch=6,validation_data=(X_test,y_test))

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 4s 35ms/step - accuracy: 0.7590 - loss: 0.4972 - val_accuracy: 0.7792 - val_loss: 0.4708
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7745 - loss: 0.4368 - val_accuracy: 0.7857 - val_loss: 0.4844
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.7740 - loss: 0.4538 - val_accuracy: 0.7792 - val_loss: 0.4785
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7621 - loss: 0.4532 - val_accuracy: 0.7792 - val_loss: 0.4750
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7872 - loss: 0.4189 - val_accuracy: 0.7987 - val_loss: 0.4748
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7862 - loss: 0.4253 - val_accuracy: 0.7987 - val_loss: 0.4732
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.7939 - loss: 0.4165 - val_accuracy: 0.7922 - val_loss: 0.4761
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7657 - loss: 0.4386 - val_accuracy: 0.

In [62]:
def build_model(hp):
  model = Sequential()

  Counter = 0

  for i in range(hp.Int('num_layers',min_value = 1,max_value = 10)):

      if Counter == 0:

          model.add(Dense(
                          hp.Int('units' + str(i),min_value = 8,max_value = 128,step=8),
                          activation=hp.Choice('activation' + str(i) , values=['relu','tanh','sigmoid']),
                          input_dim = 8)
                    )
      else:

           model.add(Dense(
                          hp.Int('units' + str(i),min_value = 8,max_value = 128,step=8),
                          activation=hp.Choice('activation' + str(i) , values=['relu','tanh','sigmoid']),
                          input_dim = 8)
                          )
      Counter+=1

  model.add(Dense(1,activation='sigmoid'))
  model.compile(optimizer=hp.Choice('optimizer',values=['rmsprop','adam','sgd','nadam','adadelta']),
                 loss = 'binary_crossentropy',metrics=['accuracy'])

  return model

    
              

In [63]:
tuner = kt.Hyperband(build_model,
                    objective='val_accuracy',
                    max_epochs=3,directory='mydir',project_name ='final')

In [64]:
tuner.search(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

Trial 6 Complete [00h 00m 06s]
val_accuracy: 0.798701286315918

Best val_accuracy So Far: 0.798701286315918
Total elapsed time: 00h 00m 46s


In [65]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 1,
 'units0': 128,
 'activation0': 'tanh',
 'optimizer': 'rmsprop',
 'units1': 112,
 'activation1': 'tanh',
 'units2': 80,
 'activation2': 'relu',
 'units3': 48,
 'activation3': 'tanh',
 'units4': 120,
 'activation4': 'sigmoid',
 'units5': 40,
 'activation5': 'sigmoid',
 'units6': 88,
 'activation6': 'relu',
 'units7': 56,
 'activation7': 'tanh',
 'units8': 32,
 'activation8': 'tanh',
 'units9': 72,
 'activation9': 'tanh',
 'tuner/epochs': 3,
 'tuner/initial_epoch': 0,
 'tuner/bracket': 0,
 'tuner/round': 0}

In [66]:
model = tuner.get_best_models(num_models=1)[0]

C:\Users\himan\anaconda3\Lib\site-packages\keras\src\saving\saving_lib.py:415: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 6 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [67]:
model.fit(X_train,y_train,batch_size=32,epochs=100,initial_epoch=6,validation_data=(X_test,y_test))

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 3s 42ms/step - accuracy: 0.7596 - loss: 0.5062 - val_accuracy: 0.7857 - val_loss: 0.4661
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.8019 - loss: 0.4505 - val_accuracy: 0.7857 - val_loss: 0.4668
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7596 - loss: 0.4953 - val_accuracy: 0.7792 - val_loss: 0.4657
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7705 - loss: 0.4791 - val_accuracy: 0.7857 - val_loss: 0.4646
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.7659 - loss: 0.4694 - val_accuracy: 0.7727 - val_loss: 0.4676
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7806 - loss: 0.4604 - val_accuracy: 0.7857 - val_loss: 0.4650
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7719 - loss: 0.4769 - val_accuracy: 0.7857 - val_loss: 0.4685
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7505 - loss: 0.4805 - val_accuracy: 0.